In [24]:
using MyMatching

In [23]:
Pkg.checkout("MyMatching", "master")

INFO: Checking out MyMatching master...
INFO: Pulling MyMatching latest master...
INFO: No packages to install, update or remove


In [25]:
Pkg.test("MyMatching")

INFO: Testing MyMatching


one-to-one: Vector of Vectors: Error During Test
  Got an exception of type UndefVarError outside of a @test
  UndefVarError: p_count not defined
  Stacktrace:
   [1] my_deferred_acceptance(::Array{Array{Int64,1},1}, ::Array{Array{Int64,1},1}, ::Array{Int64,1}, ::Array{Int64,1}) at C:\Users\r28yn\.julia\v0.6\MyMatching\src\MyMatching.jl:56
   [2] my_deferred_acceptance(::Array{Array{Int64,1},1}, ::Array{Array{Int64,1},1}) at C:\Users\r28yn\.julia\v0.6\MyMatching\src\MyMatching.jl:78
   [3] macro expansion at C:\Users\r28yn\.julia\v0.6\MyMatching\test\runtests.jl:207 [inlined]
   [4] macro expansion at .\test.jl:860 [inlined]
   [5] macro expansion at C:\Users\r28yn\.julia\v0.6\MyMatching\test\runtests.jl:201 [inlined]
   [6] macro expansion at .\test.jl:860 [inlined]
   [7] anonymous at .\<missing>:?
   [8] include_from_node1(::String) at .\loading.jl:569
   [9] include(::String) at .\sysimg.jl:14
   [10] process_options(::Base.JLOptions) at .\client.jl:305
   [11] _start() at .\client

ERROR: LoadError: Some tests did not pass: 0 passed, 0 failed, 2 errored, 0 broken.
while loading C:\Users\r28yn\.julia\v0.6\MyMatching\test\runtests.jl, in expression starting on line 17
=============================[ ERROR: MyMatching ]==============================

failed process: Process(`'C:\Users\r28yn\AppData\Local\Julia-0.6.0\bin\julia.exe' -Cx86-64 '-JC:\Users\r28yn\AppData\Local\Julia-0.6.0\lib\julia\sys.dll' --compile=yes --depwarn=yes --check-bounds=yes --code-coverage=none --color=yes --compilecache=yes 'C:\Users\r28yn\.julia\v0.6\MyMatching\test\runtests.jl'`, ProcessExited(1)) [1]



LoadError: [91mMyMatching had test errors[39m

In [ ]:
using Matching

## 例1
```
In [5]:
prop_prefs, resp_prefs = mat2vecs.(random_prefs(5, 3, allow_unmatched = false))

Out[5]:
(Array{Int64,1}[[2,1,3],[1,2,3],[2,1,3],[1,3,2],[2,1,3]],Array{Int64,1}[[1,5,3,2,4],[4,2,1,5,3],[5,1,3,2,4]])

In [6]:
p_caps = [1, 1, 1, 2, 2]
r_caps = [2, 2, 3];

In [7]:
my_deferred_acceptance(prop_prefs, resp_prefs, p_caps, r_caps)

Out[7]:
([1,2,3,3,2,3,1],[1,5,2,4,3,5,4],[1,2,3,4,6,8],[1,3,5,8])
```

## 例2
```
In [8]:
prop_prefs, resp_prefs = mat2vecs.(random_prefs(5, 3, allow_unmatched = false))

Out[8]:
(Array{Int64,1}[[3,1,2],[3,1,2],[2,3,1],[3,1,2],[1,3,2]],Array{Int64,1}[[5,4,2,3,1],[5,2,3,4,1],[5,4,1,2,3]])

In [9]:
p_caps = [1, 1, 1, 2, 2]
r_caps = [1, 3, 3];

In [10]:
my_deferred_acceptance(prop_prefs, resp_prefs, p_caps, r_caps)

Out[10]:
([3,2,2,3,2,1,3],[5,2,4,3,4,5,1],[1,2,3,4,6,8],[1,2,5,8])
```

## 例3
```
In [11]:
prop_prefs, resp_prefs = mat2vecs.(random_prefs(7, 4))

Out[11]:
(Array{Int64,1}[[3,4],[4,3,2],[2,4,1,3],[3,2],[2,4],[3,2,4],[3]],Array{Int64,1}[[3,7,6,4,5,2],[2,4,5,7,6],[4,1,3],[4,6,2]])

In [12]:
p_caps = [1, 1, 2, 1, 1, 2, 1]
r_caps = [3, 3, 1, 2];

In [13]:
my_deferred_acceptance(prop_prefs, resp_prefs, p_caps, r_caps)

Out[13]:
([0,4,1,0,3,2,2,4,0],[0,0,3,0,6,5,4,6,2],[1,2,3,5,6,7,9,10],[1,4,7,8,10])
```

In [3]:
A = zeros(Int, 2, 3)

2×3 Array{Int64,2}:
 0  0  0
 0  0  0

In [22]:
B = fill([0],(2,2))

2×2 Array{Array{Int64,1},2}:
 [0]  [0]
 [0]  [0]

In [12]:
A[1,2] = [1]

LoadError: [91mMethodError: Cannot `convert` an object of type Array{Int64,1} to an object of type Int64
This may have arisen from a call to the constructor Int64(...),
since type constructors fall back to convert methods.[39m

In [18]:
?fill

search: fill fill! finally filt filt! filter filter! filesize filemode



```
fill(x, dims)
```

Create an array filled with the value `x`. For example, `fill(1.0, (5,5))` returns a 5×5 array of floats, with each element initialized to `1.0`.

```jldoctest
julia> fill(1.0, (5,5))
5×5 Array{Float64,2}:
 1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0
```

If `x` is an object reference, all elements will refer to the same object. `fill(Foo(), dims)` will return an array filled with the result of evaluating `Foo()` once.
